## Clustering graph using document based clustering approach

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import csv
import os
import string
import numpy as np

### Source Corpus data from disc

In [2]:
path_folder = r"C:\Titan\Data\PartitionServiceData"
data_file = r"\RUSH_05Oct2017_TopLevelTradeDependency_16Nov2017.csv"
#file_path = os.path.join(path_folder, data_file)
file_path = r".\PartitionServiceData\RUSH_05Oct2017_TopLevelTradeDependency_16Nov2017.csv"
document_list = []
def createCorpus(file_path):
    corpus = []
    with open(file_path, 'rb') as csv_file:
        data = csv.reader(csv_file, delimiter='|')
        for row in data:
            # Remove the TCN label per row
            #row1 = row[1:]
            #row1 = [string.replace(s, ',', '_') for s in row]
            #row2 = [string.replace(s, ' ', '_') for s in row1]
            #document = ' '.join(row1) 
            document = row
            corpus.append(document)
            #print(row2)
    return corpus
    #print('corpus: {}'.format(corpus))


In [57]:
documents = ["DiscountCurve_USD Funding_USD Time_0 Time_1,Time_2,BondDefaultCurve_ISIN_XS1405766541 BondRecovery_ISIN_XS1405766541",
            "DiscountCurve_USD,Funding_USD Time_0 Time_1 Time_2 BondReferenceAsset_ISIN_XS0274618247 BondDefaultCurve_ISIN_XS0274618247 BondRecovery_ISIN_XS0274618247",
             "DiscountCurve_USD,Funding_USD,Time_0,Time_1,Time_2,BondReferenceAsset_ISIN_XS1396282177 BondDefaultCurve_ISIN_XS1396282177 BondRecovery_ISIN_XS1396282177",
             "DiscountCurve_USD Funding_USD Time_0 Time_1 Time_2",
             "DiscountCurve_USD Funding_USD Time_0 Time_1 Time_2",
             "DiscountCurve_USD Funding_USD Time_0,Time_1 Time_2 Recovery_REPUBLIC_OF_SOUTH_AFRICA_USD_Senior_Restructuring_2014_Bond DefaultCurve_REPUBLIC_OF SOUTH_AFRICA_USD_Senior_Restructuring_2014_Bond",
             "DiscountCurve_USD Funding_USD Time_0 Time_1 Time_2 Recovery_REPUBLIC_OF_TURKEY_USD_Senior_Restructuring_2014_Bond DefaultCurve_REPUBLIC OF TURKEY_USD_Senior_Restructuring_2014_Bond"
             
            ]
documents1 = createCorpus(file_path)
documents1[0:20]

[['RUSH-XS1405766541',
  'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2,BondDefaultCurve,ISIN,XS1405766541,BondRecovery,ISIN,XS1405766541'],
 ['RUSH-XS0274618247',
  'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2,BondReferenceAsset,ISIN,XS0274618247,BondDefaultCurve,ISIN,XS0274618247,BondRecovery,ISIN,XS0274618247'],
 ['RUSH-XS1396282177',
  'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2,BondReferenceAsset,ISIN,XS1396282177,BondDefaultCurve,ISIN,XS1396282177,BondRecovery,ISIN,XS1396282177'],
 ['501899428', 'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2'],
 ['501899864', 'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2'],
 ['501901588', 'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2'],
 ['501901870', 'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2'],
 ['501903254', 'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2'],
 ['501905124', 'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2'],
 ['501905211', 'DiscountCurve,USD,Funding,USD,Time,0,Time,1,Time,2']

In [103]:
def computeJaccardMeasure(a, b):
    A = set(a)
    B = set(b)
    intersect_A_B = A & B
    union_A_B = A | B
    jaccard_measure = float(len(intersect_A_B))/float(len(union_A_B))
    return jaccard_measure

def createJaccardDiffMatrix(data):   
    tcns = data.keys()
    jaccard_dif_matrix = np.zeros((len(tcns),len(tcns)))
    for i, key1 in enumerate(tcns):
        for j, key2 in enumerate(tcns):
            x1 = data[tcns[i]]
            x2 = data[tcns[j]]
            #print("i,j = {0},{1}".format(i,j))
            #print("x1 = {0}, \nx2 = {1}\n".format(x1, x2))
            jaccard_dif_matrix[i,j] = computeJaccardMeasure(x1, x2)
    return jaccard_dif_matrix

feature_space_dict = {
'RUSH-XS1405766541':
  'DiscountCurve_USD,Funding_USD,Time_0,Time_1,Time_2,BondDefaultCurve_ISIN_XS1405766541,BondRecovery_ISIN_XS1405766541',
 'RUSH-XS0274618247':
  'DiscountCurve_USD,Funding_USD,Time_0,Time_1,Time_2,BondReferenceAsset_ISIN_XS0274618247,BondDefaultCurve_ISIN_XS0274618247,BondRecovery_ISIN,XS0274618247',
 'RUSH-XS1396282177':
  'DiscountCurve_USD,Funding_USD,Time_0,Time_1,Time_2,BondReferenceAsset_ISIN_XS1396282177,BondDefaultCurve_ISIN,XS1396282177_BondRecovery_ISIN,XS1396282177',
 '501899428':
 'DiscountCurve_USD,Funding_USD,Time_0,Time_1,Time_2',
 '501899864':
 'DiscountCurve_USD,Funding_USD,Time_0,Time_1,Time_2',
 '501901588':
 'DiscountCurve_USD,Funding_USD,Time_0,Time_1,Time_2'
}

print("Jaccard Similarity Matrix is:\n {}".format(createJaccardDiffMatrix(feature_space_dict)))
print("\nList of TCNs are: \n{}".format(feature_space_dict.keys()))



Jaccard Similarity Matrix is:
 [[ 1.          0.92307692  0.63157895  0.63157895  0.925       0.63157895]
 [ 0.92307692  1.          0.64864865  0.64864865  0.85365854  0.64864865]
 [ 0.63157895  0.64864865  1.          1.          0.61538462  1.        ]
 [ 0.63157895  0.64864865  1.          1.          0.61538462  1.        ]
 [ 0.925       0.85365854  0.61538462  0.61538462  1.          0.61538462]
 [ 0.63157895  0.64864865  1.          1.          0.61538462  1.        ]]

List of TCNs are: 
['RUSH-XS0274618247', 'RUSH-XS1405766541', '501899428', '501901588', 'RUSH-XS1396282177', '501899864']


In [106]:
x1 = set(feature_space_dict['RUSH-XS0274618247'])
x2 = set(feature_space_dict['501901588'])
computeJaccardMeasure(x1, x2)

0.631578947368421

### Vectorize the corpus data to convert the data from text to numeric format

In [41]:
vectorizer = TfidfVectorizer(stop_words='english',lowercase=False)
X = vectorizer.fit_transform(documents)


### Display the Document Term Matrix

In [42]:
X.toarray()

array([[ 0.        ,  0.        ,  0.58945468,  0.        ,  0.        ,
         0.58945468,  0.        ,  0.        ,  0.        ,  0.        ,
         0.24701675,  0.24701675,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.24701675,  0.24701675,  0.24701675],
       [ 0.50780034,  0.        ,  0.        ,  0.50780034,  0.        ,
         0.        ,  0.50780034,  0.        ,  0.        ,  0.        ,
         0.2127987 ,  0.2127987 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.2127987 ,  0.2127987 ,  0.2127987 ],
       [ 0.        ,  0.50780034,  0.        ,  0.        ,  0.50780034,
         0.        ,  0.        ,  0.50780034,  0.        ,  0.        ,
         0.2127987 ,  0.2127987 ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.2127987 ,  0.2127987 ,  0.2127987 ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  

### Now cluster the data - using kmeans

In [48]:
true_k = 2
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=2, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

### Display the computed clusters

In [49]:
model.cluster_centers_.argsort()

array([[ 9,  1,  2,  4,  5, 16,  7,  8, 15, 14, 13, 12, 17, 19, 10, 18, 11,
         6,  3,  0],
       [ 0,  3,  6, 16, 14, 12,  8, 15, 13,  9,  7,  4,  1,  5,  2, 10, 11,
        17, 18, 19]], dtype=int64)

In [45]:
model.cluster_centers_.argsort()[:, ::-1]

array([[19, 17, 11, 10, 18,  2,  5,  7,  1,  3,  4,  6,  9, 13, 15,  0,  8,
        12, 14, 16],
       [16, 14, 12,  8, 19, 17, 11, 10, 18,  5,  1,  2,  3,  4,  9,  6,  7,
        13, 15,  0]], dtype=int64)

In [50]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print "Cluster %d:" % i,
    for ind in order_centroids[i, :10]:
        print ' %s' % terms[ind],
    print

Top terms per cluster:
Cluster 0:  BondDefaultCurve_ISIN_XS0274618247  BondRecovery_ISIN_XS0274618247  BondReferenceAsset_ISIN_XS0274618247  Funding_USD  Time_1  DiscountCurve_USD  Time_2  Time_0  OF  Recovery_REPUBLIC_OF_SOUTH_AFRICA_USD_Senior_Restructuring_2014_Bond
Cluster 1:  Time_2  Time_1  Time_0  Funding_USD  DiscountCurve_USD  BondDefaultCurve_ISIN_XS1405766541  BondRecovery_ISIN_XS1405766541  BondDefaultCurve_ISIN_XS1396282177  BondRecovery_ISIN_XS1396282177  BondReferenceAsset_ISIN_XS1396282177
